### Import Required Package

In [1]:
import pandas as pd
import talib

import json

### Import Tidal

In [2]:
import tidal as td

### Initialize Plumber

In [3]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [4]:
import pandas as pd
import os
import numpy as np
os.chdir('/home/jovyan/individualized-indicator')
test_1 = pd.read_pickle('./emb/2023_03_01/top_8_2023-06-01_2024-06-28.pkl')
df_reset = test_1.reset_index()
unique_values, indices, counts = np.unique(df_reset.iloc[:,0], return_index=True, return_counts=True)
sorted_unique_values = unique_values[np.argsort(indices)]
modified_arr = np.array(sorted_unique_values)
# df_reset.iloc[:,0] = np.array([s[:-3] for s in df_reset.iloc[:,0]]) 
df_reset.iloc[:,0] = np.array(df_reset.iloc[:,0])
multi_index_df = df_reset.set_index(['instrument', 'datetime'])
multi_index_df = multi_index_df.dropna()

DEFAULT_STOCKS = list(modified_arr.astype(object))
start_date = "2023-06-01"
end_date = "2024-06-28"

quote_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in DEFAULT_STOCKS]),
    },
)
quote_data.index.set_levels(
    pd.to_datetime(quote_data.index.levels[1]),
    level=1,
    inplace=True,
)
quote_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

instruments = quote_data.index.get_level_values('instrument').unique()

quote_data['new'] = multi_index_df.iloc[:,5]

quote_data

/tmp/ipykernel_21773/3292540928.py:28: FutureWarning: inplace is deprecated and will be removed in a future version.
  quote_data.index.set_levels(


open      high       low     close   volume  \
instrument datetime                                                      
4743       2023-06-01  202.0706  203.3885  199.8742  200.3135  1394543   
           2023-06-02  202.0706  214.3706  201.6314  210.4170  5346699   
           2023-06-05  210.8563  213.4920  206.4635  207.3420  2717866   
           2023-06-06  208.6599  211.7349  206.9028  210.4170  1998439   
           2023-06-07  211.7349  213.4920  210.4170  210.8563  1515098   
...                         ...       ...       ...       ...      ...   
6829       2024-06-24  115.5000  118.0000  114.5000  117.0000   551628   
           2024-06-25  117.0000  118.0000  112.5000  115.0000   539068   
           2024-06-26  115.5000  117.0000  115.0000  116.0000   254166   
           2024-06-27  116.0000  117.5000  113.5000  116.5000   282997   
           2024-06-28  116.5000  117.5000  115.5000  117.0000   347439   

                            new  
instrument datetime              
4743       2023-06-01  0.344074  
           2023-06-02  0.453204  
           2023-06-05  0.429721  
           2023-06-06  0.458616  
           2023-06-07  0.458097  
...                         ...  
6829       2024-06-24  0.392524  
           2024-06-25  0.392321  
           2024-06-26  0.392674  
           2024-06-27  0.392798  
           2024-06-28  0.393043  

[93365 rows x 6 columns]

### Load benchmark data

In [5]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/tmp/ipykernel_21773/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2023-06-01  120.7304  120.9711  120.2969  120.5377   4231123
           2023-06-02  121.1638  122.3195  121.1638  122.0788   8412015
           2023-06-05  122.0788  122.1269  121.5972  121.6935   6813168
           2023-06-06  121.6935  122.5603  121.5009  122.0306   6877372
           2023-06-07  122.3195  123.5235  122.3195  123.3790  12723827
...                         ...       ...       ...       ...       ...
           2024-06-24  186.5000  186.5000  183.5500  184.0000  14671365
           2024-06-25  181.9500  183.9000  180.8000  183.9000  13774562
           2024-06-26  184.9500  186.1000  184.2500  185.3500   9269483
           2024-06-27  183.6000  185.3000  183.4000  185.3000   6508242
           2024-06-28  185.3000  187.4000  184.6500  186.4500   4455951

[263 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [6]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=10000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True)

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [7]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [8]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [9]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [10]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [11]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 263/263 [00:33<00:00,  7.87it/s, cash=9.21e+4, pnl=2.71e+6, position_cost=9.47e+6, value=1.23e+7] 


### Metric - AccountInfo

In [12]:
tidal.metrics['AccountInfo'].report

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown
datetime,,,,,,,
2023-06-01,1.000000e+07,0.000000,0.000000e+00,0.0000,0.000000e+00,1.000000e+07,0.000000
2023-06-02,8.451480e+05,2999.519644,9.151852e+06,105445.9620,-5.088896e+04,9.946112e+06,-0.005389
2023-06-05,8.451480e+05,2999.519644,9.151852e+06,105445.9620,-1.015427e+05,9.895458e+06,-0.010454
2023-06-06,1.520906e+05,15804.314576,9.764443e+06,163358.4304,-7.627705e+04,9.840256e+06,-0.015974
2023-06-07,1.510969e+05,23324.901617,9.767454e+06,156625.5152,4.342074e+04,9.961972e+06,-0.003803
...,...,...,...,...,...,...,...
2024-06-24,9.214574e+04,197539.056640,9.466335e+06,419804.0996,2.338665e+06,1.189715e+07,-0.040619
2024-06-25,9.214574e+04,197539.056640,9.466335e+06,419804.0996,2.323165e+06,1.188165e+07,-0.041869
2024-06-26,9.214574e+04,197539.056640,9.466335e+06,419804.0996,2.652665e+06,1.221115e+07,-0.015298


### Metric - PositionInfo

In [13]:
tidal.metrics['PositionInfo'].report

quantity       price  commission   slip_cost  \
instrument datetime                                                   
1326       2023-06-06   30000.0   68.279386  671.357065   4088.5860   
2063       2023-06-07   49000.0   41.865163  672.344055  -4941.0032   
           2023-06-08   49000.0   41.865163  672.344055  -4941.0032   
           2023-06-09   49000.0   41.865163  672.344055  -4941.0032   
           2023-06-12   49000.0   41.865163  672.344055  -4941.0032   
...                         ...         ...         ...         ...   
9914       2023-08-11    8000.0  216.655700  568.071245  11606.4000   
           2023-08-14    8000.0  216.655700  568.071245  11606.4000   
           2023-08-15    8000.0  216.655700  568.071245  11606.4000   
           2023-08-16    8000.0  216.655700  568.071245  11606.4000   
           2023-08-17    8000.0  216.655700  568.071245  11606.4000   

                               pnl  
instrument datetime                 
1326       2023-06-06    7746.4140  
2063       2023-06-07   68214.7032  
           2023-06-08  120189.0032  
           2023-06-09  183457.8032  
           2023-06-12  178940.0032  
...                            ...  
9914       2023-08-11  -61245.6000  
           2023-08-14   -5245.6000  
           2023-08-15  -69245.6000  
           2023-08-16 -149245.6000  
           2023-08-17 -157245.6000  

[1309 rows x 5 columns]

In [14]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl
instrument,datetime,,,,,


### Metric - AdditionalInfo

In [15]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2023-06-01  0.226663
           2023-06-02  0.298821
           2023-06-05  0.313675
           2023-06-06  0.311991
           2023-06-07  0.311321
...                         ...
9958       2024-06-24  0.540087
           2024-06-25  0.539773
           2024-06-26  0.541311
           2024-06-27  0.540410
           2024-06-28  0.542090

[93365 rows x 1 columns]

### Trade Report

In [16]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
3551,2.0,0.0,2.0,1.0,1.000000,1.420508e+06,0.000000,2.753936e+06,1.420508e+06
3376,1.0,2.0,3.0,1.0,0.333333,7.165400e+05,-121674.145144,1.744200e+06,5.948659e+05
6223,1.0,0.0,1.0,0.0,1.000000,5.358491e+05,0.000000,1.008707e+04,5.358491e+05
3017,3.0,0.0,3.0,1.0,1.000000,4.539110e+05,0.000000,1.344149e+06,4.539110e+05
2385,1.0,0.0,1.0,1.0,1.000000,2.601420e+05,0.000000,1.962858e+06,2.601420e+05
2376,1.0,0.0,1.0,0.0,1.000000,1.852280e+05,0.000000,6.474724e+03,1.852280e+05
8114,2.0,2.0,4.0,1.0,0.500000,3.114859e+05,-198226.675031,1.713304e+06,1.132592e+05
2063,2.0,1.0,3.0,0.0,0.666667,1.952531e+05,-124897.634218,2.069319e+04,7.035542e+04
3675,1.0,0.0,1.0,0.0,1.000000,4.547209e+04,0.000000,5.803933e+03,4.547209e+04


### Traded instruments

In [17]:
tidal.account.trades.keys()

dict_keys(['4770', '2337', '2393', '6811', '3227', '2063', '1326', '6288', '2207', '5340', '6223', '2912', '3675', '3037', '8114', '2376', '9914', '3551', '4979', '2301', '3017', '2385', '3376'])

### Trading History

In [18]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [19]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [20]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:44943
 * Running on http://10.136.2.3:44943
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:52] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:52] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /Tidal_Logo.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [08/Jul/2024 14:01:53] "GET /image